In [2]:
import pymatgen as mg
print('hello world')

hello world


In [3]:
si = mg.Element("Si")
print('Atomic mass of Si is {}'.format(si.atomic_mass))
print("Si has a melting point of {}".format(si.melting_point))
print("Ionic radii for Si: {}".format(si.ionic_radii))

Atomic mass of Si is 28.0855 amu
Si has a melting point of 1687.0 K
Ionic radii for Si: {4: 0.54}


A Composition is essentially an *immutable* mapping of Elements/Species with amounts, and useful properties like molecular weight, get_atomic_fraction, etc. Note that you can conveniently either use an Element/Specie object or a string as keys (this is a feature).

In [5]:
comp = mg.Composition("Fe2O3")
print("Weight of Fe2O3 is {}".format(comp.weight))
print("Amount of Fe in Fe2O3 is {} (number of Fe atoms per molecule)".format(comp["Fe"]))
print("Atomic fraction of Fe is {}".format(comp.get_atomic_fraction("Fe")))
print("Weight fraction of Fe is {}".format(comp.get_wt_fraction("Fe")))

Weight of Fe2O3 is 159.6882 amu
Amount of Fe in Fe2O3 is 2.0 (number of Fe atoms per molecule)
Atomic fraction of Fe is 0.4
Weight fraction of Fe is 0.699425505453753 


A Lattice represents a Bravais lattice. Convenience static functions are provided for the creation of common lattice types from a minimum number of arguments.

In [7]:
# Creates cubic Lattice with lattice constant 4.2
lattice = mg.Lattice.cubic(4.2)
print(lattice.parameters)

(4.2, 4.2, 4.2, 90.0, 90.0, 90.0)


A Structure object represents a crystal structure (lattice + basis). A Structure is essentially a list of PeriodicSites with the same Lattice. Let us now create a CsCl structure.

In [8]:
structure = mg.Structure(lattice, ["Cs", "Cl"], [[0, 0, 0], [0.5, 0.5, 0.5]])
print("Unit cell vol = {}".format(structure.volume))
print("First site of the structure is {}".format(structure[0]))

Unit cell vol = 74.08800000000001
First site of the structure is [0. 0. 0.] Cs


The Structure object contains many useful manipulation functions. Since Structure is essentially a list, it contains a simple pythonic API for manipulation its sites. Some examples are given below. Please note that there is an immutable version of Structure known as IStructure, for the use case where you really need to enforce that the structure does not change. Conversion between these forms of Structure can be performed using from_sites().

In [9]:
structure.make_supercell([2, 2, 1]) #Make a 3 x 2 x 1 supercell of the structure
del structure[0] #Remove the first site
structure.append("Na", [0,0,0]) #Append a Na atom.
structure[-1] = "Li" #Change the last added atom to Li.
structure[0] = "Cs", [0.01, 0.5, 0] #Shift the first atom by 0.01 in fractional coordinates in the x-direction.
immutable_structure = mg.IStructure.from_sites(structure) #Create an immutable structure (cannot be modified).
print(immutable_structure)

Full Formula (Cs3 Li1 Cl4)
Reduced Formula: Cs3LiCl4
abc   :   8.400000   8.400000   4.200000
angles:  90.000000  90.000000  90.000000
Sites (8)
  #  SP       a     b    c
---  ----  ----  ----  ---
  0  Cs    0.01  0.5   0
  1  Cs    0.5   0     0
  2  Cs    0.5   0.5   0
  3  Cl    0.25  0.25  0.5
  4  Cl    0.25  0.75  0.5
  5  Cl    0.75  0.25  0.5
  6  Cl    0.75  0.75  0.5
  7  Li    0     0     0


# Basic Analyses

In [10]:
# Example: determining the symmetry if the above structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
finder = SpacegroupAnalyzer(structure)
print("The spacegroup is {}".format(finder.get_space_group_symbol()))

The spacegroup is Pmm2


Structure matching tool example.

In [12]:
from pymatgen.analysis.structure_matcher import StructureMatcher
import numpy as np
# Let's create two structures which are the same topologically, but with different elements, and one lattice is larger.
s1 = mg.Structure(lattice, ["Cs", "Cl"], [[0, 0, 0], [0.5, 0.5, 0.5]])
s2 = mg.Structure(mg.Lattice.cubic(5), ["Rb", "F"], [[0, 0, 0], [0.5, 0.5, 0.5]])
m = StructureMatcher()
print(m.fit_anonymous(s1, s2)) # Returns a mapping which maps s1 and s2 onto each other. Strict element fitting is also available.

True


# Input/Output

Pymatgen also provides IO support for various file formats in the pymatgen.io package. A convenient set of read_structure and write_structure functions are also provided which auto-detects several well-known formats.

In [13]:
# Convenient IO to various formats. Format is intelligently determined from file name and extension.
structure.to(filename="POSCAR")
structure.to(filename="CsCl.cif")

# Or if you just supply fmt, you simply get a string.
print(structure.to(fmt="poscar"))
print(structure.to(fmt="cif"))

Cs3 Li1 Cl4
1.0
8.400000 0.000000 0.000000
0.000000 8.400000 0.000000
0.000000 0.000000 4.200000
Cs Cl Li
3 4 1
direct
0.010000 0.500000 0.000000 Cs
0.500000 0.000000 0.000000 Cs
0.500000 0.500000 0.000000 Cs
0.250000 0.250000 0.500000 Cl
0.250000 0.750000 0.500000 Cl
0.750000 0.250000 0.500000 Cl
0.750000 0.750000 0.500000 Cl
0.000000 0.000000 0.000000 Li

# generated using pymatgen
data_Cs3LiCl4
_symmetry_space_group_name_H-M   'P 1'
_cell_length_a   8.40000000
_cell_length_b   8.40000000
_cell_length_c   4.20000000
_cell_angle_alpha   90.00000000
_cell_angle_beta   90.00000000
_cell_angle_gamma   90.00000000
_symmetry_Int_Tables_number   1
_chemical_formula_structural   Cs3LiCl4
_chemical_formula_sum   'Cs3 Li1 Cl4'
_cell_volume   296.35200000
_cell_formula_units_Z   1
loop_
 _symmetry_equiv_pos_site_id
 _symmetry_equiv_pos_as_xyz
  1  'x, y, z'
loop_
 _atom_site_type_symbol
 _atom_site_label
 _atom_site_symmetry_multiplicity
 _atom_site_fract_x
 _atom_site_fract_y
 _atom_site_fract